# Process the unattributed segments

Define workspace and project.

In [1]:
import arcpy
arcpy.env.overwriteOutput = True

# authentication to access secured data
from arcgis.gis import GIS
from arcgis.features import FeatureLayer

gis = GIS('pro')
db = arcpy.env.workspace = './pro/tdm_conflation.gdb'
arcpy.Describe(db)

catalogPath,./pro/tdm_conflation.gdb
dataType,Workspace


In [ ]:
key_fields = ['ABSPEED2019',
 'BASPEED2019',
 'ABSPEED2050',
 'BASPEED2050',
 'ABLANES2019',
 'BALANES2019',
 'ABLANES2050',
 'BALANES2050',
 'ABLANECAP',
 'BALANECAP',
 'ABCAPACITY2019',
 'BACAPACITY2019',
 'ABCAPACITY2050',
 'BACAPACITY2050',
 'ABDAYVOL2019',
 'BADAYVOL2019',
 'TOTDAYVOL2019',
 'ABDAYVOL2050',
 'BADAYVOL2050',
 'TOTDAYVOL2050']

In [ ]:
prj_path = r"C:\Development\sun_cloud\layers\tdm\conflation\pro\tdm_conflation.aprx"
aprx = arcpy.mp.ArcGISProject(prj_path)
map = aprx.listMaps()[0]
map.name

Function for applying definition query. 

In [ ]:
def apply_df_and_count(_aprx, map_index, query_str):
    """apply definition query to map layers and print count for each layer"""
    map = aprx.listMaps()[map_index]
    for lyr in map.listLayers():
        if lyr.supports("DEFINITIONQUERY"):
            lyr.definitionQuery = query_str
            result = arcpy.GetCount_management(lyr)
            count = int(result.getOutput(0))
            if count>0:
                print("{}:{}".format(lyr, count))
    _aprx.save() 
# apply_df_and_count(aprx, 0, None)

In [ ]:
base_query = "note = 'vc 2019 null' And conflation_type NOT IN ('no_source', 'review road representation')"

apply_df_and_count(aprx,0, base_query)

Dual to dual case.

In [ ]:
base_query = "note = 'vc 2019 null' And conflation_type NOT IN ('no_source', 'review road representation')"
sel_query = " and conflation_type in ('dual_line to dual_line')"
apply_df_and_count(aprx,0, base_query+sel_query)

In [ ]:
joined = arcpy.management.AddJoin(
    in_layer_or_view="tdm_conflation_backup_bf_dropfields",
    in_field="suncloudid",
    join_table="tdm_project",
    join_field="SUNCLOUDID",
    join_type="KEEP_COMMON",
    index_join_fields="INDEX_JOIN_FIELDS"
)

In [ ]:
# transfer over all key field values

In [ ]:
# transfer over all fields

fc = 'tdm_conflation_backup_bf_dropfields'
for k in key_fields:
    print(k)
    arcpy.management.CalculateField(
    in_table=joined,
    field="{0}.{1}".format(fc, k),
    expression="!tdm_project.{}!".format(k),
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [ ]:
# Remove definition query.

In [ ]:
apply_df_and_count(aprx, 0, None)

Dual to single line.

In [ ]:
base_query = "note = 'vc 2019 null' And conflation_type NOT IN ('no_source', 'review road representation')"
sel_query = " and conflation_type IN ('single_line to dual_line')"
apply_df_and_count(aprx,0, base_query+sel_query)

In [ ]:
def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

In [ ]:
base_query = "note = 'vc 2019 null' And conflation_type NOT IN ('no_source', 'review road representation')"
sel_query = "note = 'vc 2019 null' And conflation_type NOT IN ('no_source', 'review road representation') and conflation_type IN ('single_line to dual_line')"

In [ ]:

sel = arcpy.SelectLayerByAttribute_management('tdm_conflation_backup_bf_dropfields', 'NEW_SELECTION', base_query+sel_query)
print(arcpy.GetCount_management(sel))

In [ ]:

sc_ids = unique_values(sel, 'suncloudid')
sc_ids

In [ ]:
key_fields = [
 'ABLANES2019',
 'BALANES2019',
 'ABLANES2050',
 'BALANES2050',
 'ABCAPACITY2019',
 'BACAPACITY2019',
 'ABCAPACITY2050',
 'BACAPACITY2050',
 'ABDAYVOL2019',
 'BADAYVOL2019',
 'TOTDAYVOL2019',
 'ABDAYVOL2050',
 'BADAYVOL2050',
 'TOTDAYVOL2050']

In [ ]:

for sid in sc_ids:
# if 1==1:
    # sid = 'ADOT_146609'
    sql = "suncloudid = '{}'".format(sid)
    print(sql)

    route_ids = [row[0] for row in arcpy.da.SearchCursor(sel, 'route_id', sql)]
    print(route_ids)

    route_ids = list(set(route_ids))
    length = len(route_ids)
    if length==2:
        print(route_ids)
        # for id in route_ids:

        for f in key_fields:
            print('field='+f)
            value = [row[0] for row in arcpy.da.SearchCursor('tdm_project', f, "SUNCLOUDID ='{}'".format(sid))][0]
            print("value: {}".format(value))
            if(value):
                if(f.startswith('A')):
                    _query = "route_id = '{0}' and suncloudid = '{1}'".format(route_ids[0], sid)
                    selected = arcpy.SelectLayerByAttribute_management(sel, 'NEW_SELECTION', _query)
                    arcpy.management.CalculateField(
                        in_table=selected,
                        field=f,
                        expression=value,
                        expression_type="PYTHON3")
                        # arcpy.SelectLayerByAttribute_management
                elif(f.startswith('B')):
                    _query = "route_id = '{0}' and suncloudid = '{1}'".format(route_ids[1], sid)
                    selected = arcpy.SelectLayerByAttribute_management(sel, 'NEW_SELECTION', _query)

                    arcpy.management.CalculateField(
                        in_table=selected,
                        field=f,
                        expression=value,
                        expression_type="PYTHON3")
                        # arcpy.SelectLayerByAttribute_managemen

    else:
        _query = "suncloudid = '{0}'".format(sid)
        selected = arcpy.SelectLayerByAttribute_management(sel, 'NEW_SELECTION', _query)
        arcpy.management.CalculateField(
            in_table=selected,
            field='flag',
            expression="'{}'".format(length),
            expression_type="PYTHON3")
    

       

In [ ]:
arcpy.management.CalculateField(
    in_table="tdm_conflation_backup_bf_dropfields",
    field="TOTDAYVOL2019",
    expression="(!ABDAYVOL2019!,!BADAYVOL2019!)",
    expression_type="PYTHON3",
    code_block="""def sum_null(*args):
    args = [a for a in args if not a is None]
    return sum(args) if args else None""",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [ ]:
#vs_2019
arcpy.management.CalculateField(
    in_table="tdm_conflation_backup_bf_dropfields",
    field="volume_2019",
    expression="max_null(!ABDAYVOL2019!,!BADAYVOL2019!,!TOTDAYVOL2019!)",
    expression_type="PYTHON3",
    code_block="""def max_null(*args):
    args = [a for a in args if not a is None]
    return max(args) if args else None""",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [5]:
fc = 'tdm_conflation_0320backup'
target_fields = [
    ('suncloudid', 'sun_cloud_id', 'Segment ID'),
    ('volume_2019', 'volume_2019', 'Daily Volume 2019'),
    ('volume_2050','volume_2050','Daily Volume 2050'),
    ('speed_2019','speed_2019','Peak Hour Speed 2019'),
    ('speed_2050','speed_2050','Peak Hour Speed 2050'),
    ('lanes_2019','lanes_2019','Lanes 2019'),
    ('lanes_2050','lanes_2050','Lanes 2050'),
    ('capacity_2019','capacity_2019','Capacity 2019'),
    ('capacity_2050','capacity_2050','Capacity 2050'),
    ('vc_2019','vc_2019','Volume/Capacity Ratio 2019'),
    ('vc_2050','vc_2050','Volume/Capacity Ratio 2050')

]
for current, new, alias in target_fields:
    arcpy.management.AlterField(fc, current, new, alias)

In [3]:
# drop and rename fields
required_fields = ['OBJECTID', 'route_id', 'functional_class', 'sun_cloud_id',
'volume_2019','volume_2050', 'lanes_2019','lanes_2050','capacity_2019', 'capacity_2050',
'vc_2019', 'vc_2050']

all_fields = arcpy.ListFields(fc)
f_names = [f.name for f in all_fields if f.required == False]

to_delete = [i for i in f_names if i not in required_fields]
to_delete

['conflation_type',
 'ABDAYVOL2019',
 'BADAYVOL2019',
 'TOTDAYVOL2019',
 'ABDAYVOL2050',
 'BADAYVOL2050',
 'TOTDAYVOL2050',
 'ABSPEED2019',
 'BASPEED2019',
 'ABSPEED2050',
 'BASPEED2050',
 'ABLANES2019',
 'BALANES2019',
 'ABLANES2050',
 'BALANES2050',
 'ABLANECAP',
 'BALANECAP',
 'ABCAPACITY2019',
 'BACAPACITY2019',
 'ABCAPACITY2050',
 'BACAPACITY2050',
 'speed_2019',
 'speed_2050',
 'note',
 'flag',
 'SUNCLOUDID_1']

In [4]:
arcpy.DeleteField_management(fc, to_delete)

<Result './pro/tdm_conflation.gdb\\tdm_conflation_backup_bf_dropfields'>